In [3]:
import rioxarray
import numpy as np
import pandas as pd
from datascience import *   
%matplotlib inline
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime 

In [4]:
HEIGHT = 90
WIDTH = 250
all_data = [[[] for _ in range(WIDTH)] for _ in range(HEIGHT)]
cordinates = {}
tmp_name = ['AWS', 'CAPE', 'CIN', 'EWSS', 'IE', 'ISOR', 'KX', 'PEV', 'R250', 'R500', 'R850', 'SLHF', 'SLOR', 'SSHF', 'TCLW', 'TCW', 'TCWV', 'U250', 'U850', 'V250', 'V850']
file_name = [['Precipitation','AWS']]
for i in range(1,len(tmp_name)):
    file_name.append(['ERA5',tmp_name[i]])
# file_name

Hàm lấy các tọa độ có dữ liệu AWS

In [5]:
def get_AWS_cordinates(hour_list, day_list, month_list, year_list, cordinates):
    for year in year_list:
        for month in month_list:
            for day in day_list:
                for hour in hour_list:
                    file_path = f'DATA_SV/Precipitation/AWS/{year}/{month:02}/{day:02}/AWS_{year}{month:02}{day:02}{hour:02}0000.tif'
                    path = Path(file_path)
                    if not path.is_file():
                        continue
                    dataset = rioxarray.open_rasterio(file_path)
                    data = dataset[0].values
                    for i in range(HEIGHT):
                        for j in range(WIDTH):
                            if data[i][j] != -np.inf:
                                cordinates[(i,j)] = True
                                # print(i,j)
    cordinates = sorted(cordinates)

Hàm đọc dữ liệu

In [6]:

def read(hour, day, month, year, file, cordinates, all_data):
    data = []
    for i in range(len(file)):
        name1 = file[i][0]
        name2 = file[i][1]
        file_path = f'DATA_SV/{name1}/{name2}/{year}/{month:02}/{day:02}/{name2}_{year}{month:02}{day:02}{hour:02}0000.tif'
        # print(file_path)
        path = Path(file_path)
        if not path.is_file():
            return
        # print(file_path)
        dataset = rioxarray.open_rasterio(file_path)
        data.append(dataset[0].values)
        
    # print(len(data))
    if(data != []): 
        for pos in cordinates:
            i = pos[0]
            j = pos[1]
            check = True
            tmp = [datetime(year,month,day,hour)]
            for z in range(len(data)):
                if(data[z][i][j] == -np.inf or data[z][i][j] == np.inf or data[z][i][j] == np.nan):
                    check = False
                    break
                tmp.append(data[z][i][j])
            if check :
                all_data[i][j].append(tmp)
                # print(i,j,tmp)
                
            

Đọc dự liệu trong khoảng thời gian định sẵn tại đây. dữ liệu all_data[i][j][z] là tại vị trí i, j có tập giá trị tương ứng tại cùng thời điểm

In [ ]:

for i in range(HEIGHT):
    for j in range(WIDTH):
        all_data[i][j] = []
cordinates.clear
# hour = [0]
hour = [i for i in range(24)]
day = [1]
# day = [i for i in range(27,28)]
day = [i for i in range(1,32)]
month = [10]
month = [4,10]
year = [2020] 
year = [2019,2020]
get_AWS_cordinates(hour, day, month, year, cordinates)
cordinates_list = list(cordinates)
for i in year:
    for j in month:
        for k in day:
            for l in hour:
                # arr = [file_name[0],file_name[1]]
                # arr = [file_name[0]]
                read(l, k, j, i, file_name, cordinates_list, all_data)

Kiểm tra dữ liệu vừa đọc (không chạy nếu ko cần)

In [9]:
for (i,j) in cordinates_list:
    for z in range(1,len(all_data[i][j])):
        data = all_data[i][j][z]
        print(data)
        # print(len(data))
        # tmp = np.array(data)
        # print(tmp.shape)

[datetime.datetime(2020, 10, 1, 1, 0), 1.3, 175.0, 115.67191, -121.28516, -4.4376124e-05, 0.58824843, 37.648827, -6.9569098e-05, 64.894165, 82.539215, 89.586487, -256706.0, 0.019672839, -36872.0, 0.22268677, 62.602509, 62.323212, -5.8593445, -0.81410217, -0.8301239, 3.771225]
[datetime.datetime(2020, 10, 1, 2, 0), 0.60000002, 134.75, 9999.0, -187.67188, -6.4929482e-05, 0.58824843, 36.808929, -0.00012970809, 70.185623, 85.335068, 89.11203, -408081.0, 0.019672839, -48478.0, 0.15148926, 62.852478, 62.61615, -5.5141449, -0.77455139, -1.5427246, 3.3406982]
[datetime.datetime(2020, 10, 1, 3, 0), 0.0, 171.5, 9999.0, -236.19141, -6.0467282e-05, 0.58824843, 36.034546, -0.00022201263, 76.63797, 88.518127, 88.222641, -610104.0, 0.019672839, -88277.0, 0.16964722, 63.341171, 63.042343, -5.0198975, -1.0900269, -2.1270905, 2.6020508]
[datetime.datetime(2020, 10, 1, 4, 0), 0.0, 202.25, 9999.0, -321.34766, -0.00010191533, 0.58824843, 36.065269, -0.00021349755, 85.466095, 89.573853, 89.643921, -566723.0

Xây dựng bộ dữ liệu X,y để cho vào mô hình. Hiện tại một cặp X,y tương ứng 6 tiếng đoán 6 tiếng sau, có thể xây dựng tùy chỉnh lại nếu thích.

In [67]:
X = []
y = []
for (i,j) in cordinates_list:
    for z in range(6,len(all_data[i][j])-5):
        data = all_data[i][j][z:z+6]  
        pre_data = all_data[i][j][z-6:z]
        check = True
        for k in range(z-5,z+6):
            # print(all_data[i][j][k])
            if (all_data[i][j][k][0] - all_data[i][j][k-1][0]).total_seconds() == 3600:
                check = False
                break
        if check:
            continue
        
        # print(len(pre_data),len(data))
        pre_data = np.array(pre_data)
        data = np.array(data)
        
        # print(pre_data.shape,data.shape)
        
        pre_data = pre_data[:,2:].reshape(-1)
        data = data[:,1].reshape(-1)

        
        X.append(pre_data)
        y.append(data)
        # print(pre_data[2:],data[1])

Kiểm tra lại bộ X,y vừa xây

In [68]:
# for data in X:
    # print(data)
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

(3241, 120)
(3241, 6)
